In [1]:
pip install librosa soundfile numpy pandas



   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ------------------------------ --------- 0.8/1.0 MB 4.0 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 3.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ----------- ---------------------------- 0.8/2.8 MB 4.0 MB/s eta 0:00:01
   -------------- ------------------------- 1.0/2.8 MB 4.0 MB/s eta 0:00:01
   ----------------------------- ---------- 2.1/2.8 MB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 3.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/30.3 MB ? eta -:--:--
    --------------------------------------- 0.5/30.3 MB 3.8 MB/s eta 0:00:08
   - -------------------------------------- 1.3/30.3 MB 3.5 MB/s eta 0:00:09
   -- ------------------------------------- 2.1/30.3 MB 3.6 MB/s eta 0:00:08
   --- ------------------------------------ 2.9/30.3 MB 3.7 MB/s eta 0:00:08
   ---- -------------------


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
# At the top of your notebook:
import os, sys
from pathlib import Path

# 1) Figure out your project root (two levels up from notebooks/Audio)
project_root = Path().resolve().parents[1]
# 2) (Optional) add it to sys.path so `from src.audio_utils` works
sys.path.insert(0, str(project_root))

# 3) Now set BASE relative to the project root
BASE = project_root / "data" / "raw" / "audio_raw"

from src.audio_utils import extract_audio_stats


In [4]:
import sys, os
# go up two levels from notebooks/Audio → project root
proj_root = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
if proj_root not in sys.path:
    sys.path.insert(0, proj_root)

from src.audio_utils import extract_audio_stats


In [12]:
import os, pandas as pd
from src.audio_utils import extract_audio_stats

BASE = "../data/raw/audio_raw"
folders = {"everyday": 0, "snore": 1}


In [13]:
rows = []
for name, label in folders.items():
    for fn in os.listdir(f"{BASE}/{name}"):
        if not fn.endswith(".wav"): continue
        stats = extract_audio_stats(f"{BASE}/{name}/{fn}")
        rows.append({"label": label, "file": fn, **stats})
df = pd.DataFrame(rows)
thresh = (df[df.label==0].rms_mean.mean() + df[df.label==1].rms_mean.mean())/2
print("Threshold:", thresh)
df.to_csv("../data/processed/audio_features.csv", index=False)


FileNotFoundError: [WinError 3] The system cannot find the path specified: '../data/raw/audio_raw/everyday'

In [7]:
import numpy as np
import librosa

def extract_audio_stats(path):
    """
    Load `path`, compute:
      - mean RMS energy
      - RMS energy std (how “bursty” it is)
    """
    y, sr = librosa.load(path, sr=None)
    rms = librosa.feature.rms(y=y)[0]          # frame-wise RMS
    return {
        "rms_mean": np.mean(rms),
        "rms_std":  np.std(rms),
    }


In [14]:
import os
import numpy as np
import pandas as pd
from extract_audio_stats import extract_audio_stats  # or wherever you put that fn

# 1) Set this to wherever you unzipped the Snoring_Dataset
BASE_DIR = r"C:\Users\Malak\Downloads\archive\Snoring Dataset"

rows = []
for label in ["0","1"]:                   # "0" = everyday noise, "1" = snoring
    folder = os.path.join(BASE_DIR, label)
    for fn in os.listdir(folder):
        if not fn.lower().endswith(".wav"):
            continue
        file_path = os.path.join(folder, fn)
        stats = extract_audio_stats(file_path)
        rows.append({
            "label": int(label),
            "file_path": file_path,
            **stats
        })

df_audio = pd.DataFrame(rows)
print(df_audio.head())
print(df_audio.groupby("label")["rms_mean"].describe())


ModuleNotFoundError: No module named 'extract_audio_stats'

In [15]:
import os, pandas as pd
from src.audio_utils import extract_audio_stats

# Use the project_root that was already set up
project_root = Path().resolve().parents[1]
BASE = project_root / "data" / "raw" / "audio_raw"
processed_dir = project_root / "data" / "processed"

# Ensure the processed directory exists
os.makedirs(processed_dir, exist_ok=True)

folders = {"everyday": 0, "snore": 1}

rows = []
for name, label in folders.items():
    folder_path = BASE / name
    if not folder_path.exists():
        print(f"Warning: Directory {folder_path} does not exist")
        continue
        
    for fn in os.listdir(folder_path):
        if not fn.endswith(".wav"): 
            continue
        file_path = folder_path / fn
        stats = extract_audio_stats(str(file_path))
        rows.append({"label": label, "file": fn, **stats})

df = pd.DataFrame(rows)
if not df.empty:
    thresh = (df[df.label==0].rms_mean.mean() + df[df.label==1].rms_mean.mean())/2
    print("Threshold:", thresh)
    
    # Save using absolute path
    output_path = processed_dir / "audio_features.csv"
    df.to_csv(output_path, index=False)
else:
    print("No audio files were processed. Please check if the audio files exist in the correct directories.")

Threshold: 0.09155986498534913


SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (3560050746.py, line 1)